<a href="https://colab.research.google.com/github/prakhar-kt/HuggingFaceNotebooks/blob/main/AIPoweredMeetingMinutesGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2

In [16]:
import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive, userdata
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig, AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import torch

In [3]:
AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [4]:
audio_file = "/content/drive/MyDrive/LLMEngineering/Datasets/MeetingBank/Denver.mp3"

In [5]:
openai_api_key = userdata.get("OPENAI_API_KEY")
openai = OpenAI(api_key=openai_api_key)

In [6]:
audio_file = open(audio_file, "rb")
transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
print(transcription)

The Denver City Council honors and acknowledges that the land on which we reside is the traditional territory of the Ute, Cheyenne, and Arapaho peoples. We also recognize the 48 contemporary tribal nations that are historically tied to the lands that make up the state of Colorado. Right. The second Monday of most months, the Denver City Council celebrates and promotes Imagine 2020 Denver's cultural plan by featuring a poet, a musician, or performer. Before the council meeting built on the seven revelations, Imagine 2020 strives to integrate the arts into daily life, make the arts more accessible, and ensure that Denver thrives.



In [7]:
system_message = "You are helpful assistant that generates minutes of the meeting, \
                 with summary, key discussion point, takeways and actin items in \
                Markdown Format"
user_prompt = f"Below is the extract transcript of denver council meeting.\
                Please write minutes in markdown, including a summary, discussion points, takeaways, \
                and action items. \n{transcription} "

In [8]:
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
]

In [9]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [10]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
model = AutoModelForCausalLM.from_pretrained(LLAMA, quantization_config=quant_config, device_map="auto")
outputs = model.generate(inputs=inputs, streamer=streamer, max_new_tokens=1000, use_cache=True)

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


assistant

**Meeting Minutes**

**Summary**
----------

The Denver City Council meeting honored and acknowledged the traditional territory of the Ute, Cheyenne, and Arapaho peoples, as well as the 48 contemporary tribal nations tied to the lands of Colorado.

**Key Discussion Points**
-------------------------

*   Honoring and acknowledging the traditional territory and contemporary tribal nations of Colorado
*   Celebrating and promoting Imagine 2020 Denver's cultural plan
*   Integrating the arts into daily life and making them more accessible

**Takeaways**
------------

*   The Denver City Council recognizes the importance of acknowledging and honoring the traditional territory and contemporary tribal nations of Colorado
*   Imagine 2020 Denver's cultural plan aims to integrate the arts into daily life and make them more accessible
*   The council celebrates and promotes the cultural plan by featuring a poet, musician, or performer on the second Monday of most months

**Action Ite

In [12]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True, skip_prompt=True)
display(Markdown(response))

system

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are helpful assistant that generates minutes of the meeting,                  with summary, key discussion point, takeways and actin items in                 Markdown Formatuser

Below is the extract transcript of denver council meeting.                Please write minutes in markdown, including a summary, discussion points, takeaways,                 and action items. 
The Denver City Council honors and acknowledges that the land on which we reside is the traditional territory of the Ute, Cheyenne, and Arapaho peoples. We also recognize the 48 contemporary tribal nations that are historically tied to the lands that make up the state of Colorado. Right. The second Monday of most months, the Denver City Council celebrates and promotes Imagine 2020 Denver's cultural plan by featuring a poet, a musician, or performer. Before the council meeting built on the seven revelations, Imagine 2020 strives to integrate the arts into daily life, make the arts more accessible, and ensure that Denver thrives.assistant

**Meeting Minutes**
===============

**Summary**
----------

The Denver City Council meeting honored and acknowledged the traditional territory of the Ute, Cheyenne, and Arapaho peoples, as well as the 48 contemporary tribal nations tied to the lands of Colorado.

**Key Discussion Points**
-------------------------

*   Honoring and acknowledging the traditional territory and contemporary tribal nations of Colorado
*   Celebrating and promoting Imagine 2020 Denver's cultural plan
*   Integrating the arts into daily life and making them more accessible

**Takeaways**
------------

*   The Denver City Council recognizes the importance of acknowledging and honoring the traditional territory and contemporary tribal nations of Colorado
*   Imagine 2020 Denver's cultural plan aims to integrate the arts into daily life and make them more accessible
*   The council celebrates and promotes the cultural plan by featuring a poet, musician, or performer on the second Monday of most months

**Action Items**
----------------

*   Continue to acknowledge and honor the traditional territory and contemporary tribal nations of Colorado in future council meetings
*   Implement Imagine 2020 Denver's cultural plan by integrating the arts into daily life and making them more accessible to the community
*   Feature a poet, musician, or performer on the second Monday of most months to promote the cultural plan